In [1]:
import nltk
import itertools
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [2]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
def arrangeTweet(tweet):
    if ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet)) in english_vocab:
        return ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    else:
        return ''.join(''.join(s)[:1] for _, s in itertools.groupby(tweet))

In [3]:
oldfile=open("train_pos.txt",'rb')
sentences=[]
i=0
for line in oldfile:
    if i%10000==0:
        print(i)
    i=i+1
    
    line = line.decode('utf8')
    line=arrangeTweet(line)
    #if not (line in sentences):
    sentences.append(line)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [4]:
oldfile=open("train_neg.txt",'rb')
i=0
for line in oldfile:
    if i%10000==0:
        print(i)
    i=i+1
    
    line = line.decode('utf8')
    line=arrangeTweet(line)
    #if not (line in sentences):
    sentences.append(line)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [5]:
indexes=np.arange(0,200000)
np.random.shuffle(indexes)

In [6]:
y=[0]*100000
y2=[1]*100000
y=y+y2
y=np.array(y)
y=y[indexes]

In [7]:
def scikit_TFIDF(m,n,Total_clean_train,Total_clean_test):
    vectorizer = CountVectorizer(min_df=1,max_features=m,ngram_range=(1,n))
    analyze = vectorizer.build_analyzer
    X_train = vectorizer.fit_transform(Total_clean_train).toarray()
    X_test = vectorizer.transform(Total_clean_test).toarray()
    transformer = TfidfTransformer()
    tfidf_train=transformer.fit_transform(X_train).toarray()
    tfidf_test=transformer.transform(X_test).toarray()
    return tfidf_train,tfidf_test

In [8]:
testfile=open("test_data.txt",'rb')
i=0
sentences_test=[]
for line in testfile:
    if i%5000==0:
        print(i)
    i=i+1
    line=line.decode('utf8')
    line=arrangeTweet(line)
    sentences_test.append((line[line.find(',')+1:]))

0
5000


In [9]:
X_train,X_test=scikit_TFIDF(12000,2,sentences,sentences_test)

In [10]:
X_train=X_train[indexes]

In [11]:
test_x=X_train[170000:]
test_y=y[170000:]

In [12]:
from sklearn import linear_model
logisticReg = linear_model.LogisticRegression(multi_class='ovr')
logisticReg.fit(X_train[0:169999], y[0:169999])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
Y_predict=logisticReg.predict(test_x)

In [14]:
print(classification_report(Y_predict,test_y))

             precision    recall  f1-score   support

          0       0.84      0.81      0.82     15621
          1       0.80      0.83      0.81     14379

avg / total       0.82      0.82      0.82     30000



In [24]:
vectorizer = CountVectorizer(min_df=1,max_features=12)
corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?',]
X = vectorizer.fit_transform(corpus)
print(X.toarray())                             

[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]
